# 통제 변수 추가하기

In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

## 1. Industry 추가하기

In [2]:
#뉴스 데이터 불러오기
news_df = pd.read_csv('/Users/parkjunhyeong/Desktop/박준형/02. 대내 및 대외활동/01. 대내활동/03. Biz&AI 랩/02. 소스/01. 데이터/06. 회귀분석 데이터/Composed_Data_ver1.1.csv', index_col = 0)
news_df.reset_index(drop = True, inplace = True)
news_df

,URL,Publish Date,Title,Main Text,Ticker,Description,News Type,Sentiment,Investor Reaction
0,https://www.nytimes.com/2019/01/01/business/me...,2019-01-01,Netflix Blocks Show in Saudi Arabia Critical o...,Netflix has blocked an episode of its show “Pa...,['NFLX'],['Netflix Inc'],ESG,Negative,0.064605
1,https://www.nytimes.com/2019/01/02/business/ju...,2019-01-02,Julie Sweet of Accenture Could See Her Future....,Why did you go to law school?\n \n I decided t...,['ACN'],['Accenture Plc Class A'],Non-ESG,Positive,-0.001291
2,https://www.nytimes.com/2019/01/02/business/te...,2019-01-02,"Tesla Reports Record Output, but Cuts Prices, ...",Tesla shares ended trading nearly 7 percent lo...,['TSLA'],['Tesla Inc'],Financial Performance,Negative,-0.010247
3,https://www.nytimes.com/2019/01/03/business/de...,2019-01-03,Bristol-Myers to Acquire Celgene in Deal Worth...,Bristol-Myers Squibb said on Thursday that it ...,['BMY'],['Bristol Myers Squibb'],Financial Performance,Neutral,-0.056534
4,https://www.nytimes.com/2019/01/03/business/de...,2019-01-03,Was Apple’s Warning a Surprise? Not to Its Sup...,"Apple’s stock fell 10 percent on Thursday, its...",['AAPL'],['Apple Inc'],Financial Performance,Negative,-0.027306
...,...,...,...,...,...,...,...,...,...
4556,https://www.nytimes.com/2022/10/18/technology/...,2022-10-18,Amazon Labor Union Loses Election at Warehouse...,Mr. Smalls said the election “wasn’t free and ...,['AMZN'],['Amazon.Com Inc.'],ESG,Positive,0.010129
4557,https://www.nytimes.com/2022/10/20/technology/...,2022-10-20,Texas Sues Google for Collecting Biometric Dat...,"José Castañeda, a Google spokesman, said in a ...",['GOOG'],['Alphabet Inc Class C'],ESG,Negative,-0.040082
4558,https://www.nytimes.com/2022/10/25/technology/...,2022-10-25,Microsoft Reports Slowest Revenue Growth in Fi...,"In the most recent quarter, softness in the gl...",['MSFT'],['Microsoft Corp'],Financial Performance,Positive,-0.036752
4559,https://www.nytimes.com/2022/10/27/technology/...,2022-10-27,"Apple’s iPhone Powers Growth, but Signs Point ...",When Apple released the 16th version of its iP...,['AAPL'],['Apple Inc'],Financial Performance,Positive,-0.003742


In [3]:
meta_df = pd.read_excel('/Users/parkjunhyeong/Desktop/박준형/02. 대내 및 대외활동/01. 대내활동/03. Biz&AI 랩/02. 소스/01. 데이터/01. 메타 데이터/Stocks in the Russell 1000 Index.xlsx', sheet_name = 'Sheet1')
meta_df

,Symbol,Description,Category2,Category3,GICS Sector,Market cap,Dividend yield,Country,Primary SICS Sector,Primary SICS Industry
0,AAPL,Apple Inc,Common stocks,Large cap,Information Technology,2234648016430,0.0064,USA,Technology & Communications,Hardware
1,MSFT,Microsoft Corp,Common stocks,Large cap,Information Technology,1808798372569,0.0104,USA,Technology & Communications,Software & IT Services
2,GOOG,Alphabet Inc Class C,Common stocks,Large cap,Communication Services,1212009730000,0.0000,USA,Technology & Communications,Internet Media & Services
3,GOOGL,Alphabet Inc Class A,Common stocks,Large cap,Communication Services,1212009730000,0.0000,USA,Technology & Communications,Internet Media & Services
4,AMZN,Amazon.Com Inc.,Common stocks,Large cap,Consumer Discretionary,994865315244,0.0000,USA,Consumer Goods,E-commerce
...,...,...,...,...,...,...,...,...,...,...
1003,UWMC,UWM Holdings Corporation Class A,Common stocks,Small cap,Financials,406406669,0.0878,USA,Analyze,NaN
1004,VTS,"Vitesse Energy, Inc.",Common stocks,Unknown,Energy,0,0.0000,USA,NaN,NaN
1005,GEHC,GE HealthCare Technologies Inc.,Common stocks,Unknown,Health Care,0,0.0000,USA,Analyze,NaN
1006,MBC,"MasterBrand, Inc.",Common stocks,Unknown,Industrials,0,0.0000,USA,Analyze,NaN


In [4]:
GICS_Sectors = []
tickers = [x[2:-2] for x in news_df['Ticker']]

for ticker in tickers:
    sector = meta_df[meta_df['Symbol'] == ticker].iloc[0,4]
    GICS_Sectors.append(sector)

In [5]:
GICS_Sectors

['Communication Services',
 'Information Technology',
 'Consumer Discretionary',
 'Health Care',
 'Information Technology',
 'Communication Services',
 'Financials',
 'Consumer Discretionary',
 'Information Technology',
 'Consumer Discretionary',
 'Consumer Discretionary',
 'Utilities',
 'Utilities',
 'Financials',
 'Communication Services',
 'Consumer Discretionary',
 'Communication Services',
 'Financials',
 'Consumer Discretionary',
 'Utilities',
 'Communication Services',
 'Communication Services',
 'Consumer Discretionary',
 'Consumer Discretionary',
 'Communication Services',
 'Information Technology',
 'Information Technology',
 'Utilities',
 'Information Technology',
 'Utilities',
 'Utilities',
 'Consumer Discretionary',
 'Communication Services',
 'Communication Services',
 'Financials',
 'Information Technology',
 'Consumer Staples',
 'Information Technology',
 'Energy',
 'Communication Services',
 'Communication Services',
 'Communication Services',
 'Utilities',
 'Utilities

In [6]:
news_df['GICS Sectors'] = GICS_Sectors

In [7]:
news_df

,URL,Publish Date,Title,Main Text,Ticker,Description,News Type,Sentiment,Investor Reaction,GICS Sectors
0,https://www.nytimes.com/2019/01/01/business/me...,2019-01-01,Netflix Blocks Show in Saudi Arabia Critical o...,Netflix has blocked an episode of its show “Pa...,['NFLX'],['Netflix Inc'],ESG,Negative,0.064605,Communication Services
1,https://www.nytimes.com/2019/01/02/business/ju...,2019-01-02,Julie Sweet of Accenture Could See Her Future....,Why did you go to law school?\n \n I decided t...,['ACN'],['Accenture Plc Class A'],Non-ESG,Positive,-0.001291,Information Technology
2,https://www.nytimes.com/2019/01/02/business/te...,2019-01-02,"Tesla Reports Record Output, but Cuts Prices, ...",Tesla shares ended trading nearly 7 percent lo...,['TSLA'],['Tesla Inc'],Financial Performance,Negative,-0.010247,Consumer Discretionary
3,https://www.nytimes.com/2019/01/03/business/de...,2019-01-03,Bristol-Myers to Acquire Celgene in Deal Worth...,Bristol-Myers Squibb said on Thursday that it ...,['BMY'],['Bristol Myers Squibb'],Financial Performance,Neutral,-0.056534,Health Care
4,https://www.nytimes.com/2019/01/03/business/de...,2019-01-03,Was Apple’s Warning a Surprise? Not to Its Sup...,"Apple’s stock fell 10 percent on Thursday, its...",['AAPL'],['Apple Inc'],Financial Performance,Negative,-0.027306,Information Technology
...,...,...,...,...,...,...,...,...,...,...
4556,https://www.nytimes.com/2022/10/18/technology/...,2022-10-18,Amazon Labor Union Loses Election at Warehouse...,Mr. Smalls said the election “wasn’t free and ...,['AMZN'],['Amazon.Com Inc.'],ESG,Positive,0.010129,Consumer Discretionary
4557,https://www.nytimes.com/2022/10/20/technology/...,2022-10-20,Texas Sues Google for Collecting Biometric Dat...,"José Castañeda, a Google spokesman, said in a ...",['GOOG'],['Alphabet Inc Class C'],ESG,Negative,-0.040082,Communication Services
4558,https://www.nytimes.com/2022/10/25/technology/...,2022-10-25,Microsoft Reports Slowest Revenue Growth in Fi...,"In the most recent quarter, softness in the gl...",['MSFT'],['Microsoft Corp'],Financial Performance,Positive,-0.036752,Information Technology
4559,https://www.nytimes.com/2022/10/27/technology/...,2022-10-27,"Apple’s iPhone Powers Growth, but Signs Point ...",When Apple released the 16th version of its iP...,['AAPL'],['Apple Inc'],Financial Performance,Positive,-0.003742,Information Technology


In [8]:
news_df['GICS Sectors'].value_counts()

Communication Services    1554
Consumer Discretionary    1072
Industrials                519
Information Technology     481
Health Care                435
Financials                 241
Consumer Staples           123
Utilities                   66
Energy                      58
Real Estate                  8
Materials                    4
Name: GICS Sectors, dtype: int64

## 2. ESG Score 추가하기

In [72]:
esg_df = pd.read_csv('/Users/parkjunhyeong/Desktop/박준형/02. 대내 및 대외활동/01. 대내활동/03. Biz&AI 랩/99. 기타/01. 비즈랩11기/ESG DATA_10기/Refinitiv_2002_2020.csv')
esg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133021 entries, 0 to 133020
Data columns (total 21 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   OrgID          133021 non-null  int64  
 1   FisYear        133021 non-null  int64  
 2   CDFlag         133021 non-null  int64  
 3   overall_score  129667 non-null  float64
 4   econ_score     129667 non-null  float64
 5   envrn_score    126001 non-null  float64
 6   lenvrnmntl     0 non-null       float64
 7   corpgov_score  129667 non-null  float64
 8   social_score   129667 non-null  float64
 9   feeddate       133021 non-null  int64  
 10  fyenddate      125845 non-null  float64
 11  Asset4Id       132762 non-null  object 
 12  OrgName        132762 non-null  object 
 13  BusTRBCId      132720 non-null  float64
 14  PrimExchg      132019 non-null  object 
 15  StatusFlag     132720 non-null  float64
 16  OAId           132720 non-null  float64
 17  tick           132582 non-nul

In [63]:
# news_df['esg_score'] = 0
# for i in tqdm(range(len(news_df))):
#     try:
#         ticker = news_df.iloc[i,4][2:-2]
#         esg_score = esg_df[esg_df['tick'] == ticker]['overall_score'].mean()
#         news_df.iloc[i,-1] = esg_score
#     except Exception as e:
#         print(i, e)

In [77]:
news_df['esg_score'] = 0

for i in tqdm(range(len(news_df))):
    try:
        if '2022' in news_df.iloc[i,1]:
            ticker = news_df.iloc[i,4][2:-2]
            news_df.iloc[i,-1] = esg_df[(esg_df['tick'] == ticker) & (esg_df['FisYear'] == 2020)]['overall_score']
        elif '2021' in news_df.iloc[i,1]:
            ticker = news_df.iloc[i,4][2:-2]
            news_df.iloc[i,-1] = esg_df[(esg_df['tick'] == ticker) & (esg_df['FisYear'] == 2020)]['overall_score']
        elif '2020' in news_df.iloc[i,1]:
            ticker = news_df.iloc[i,4][2:-2]
            news_df.iloc[i,-1] = esg_df[(esg_df['tick'] == ticker) & (esg_df['FisYear'] == 2020)]['overall_score']
        else:
            ticker = news_df.iloc[i,4][2:-2]
            news_df.iloc[i,-1] = esg_df[(esg_df['tick'] == ticker) & (esg_df['FisYear'] == 2019)]['overall_score']
    except Exception as e:
        pass

100%|██████████████████████████████████████| 4561/4561 [00:20<00:00, 224.30it/s]


## 3. firm age 추가하기

In [9]:
year_df = pd.read_excel('/Users/parkjunhyeong/Desktop/박준형/02. 대내 및 대외활동/01. 대내활동/03. Biz&AI 랩/02. 소스/01. 데이터/05. 통제변수 데이터/tickers_ver0.2_firm year.xlsx', sheet_name = 'Sheet1', index_col = 0)

In [10]:
year_df

,ticker,Description,est.year,fiem age
0,CSCO,Cisco Systems Inc,1984.0,39
1,UAL,United Continental Holdings Inc,1968.0,55
2,W,Wayfair Inc Class A,2002.0,21
3,BA,Boeing,1916.0,107
4,GILD,Gilead Sciences Inc,1987.0,36
...,...,...,...,...
215,DOV,Dover Corp,1955.0,68
216,M,Macys Inc.,1858.0,165
217,CRM,Salesforce.Com Inc,1999.0,24
218,BLK,Blackrock Inc,1988.0,35


In [11]:
news_df['Firm Age'] = 0
for i in tqdm(range(len(news_df))):
    if '2022' in news_df.iloc[i,1]:
        ticker = news_df.iloc[i,4][2:-2]
        news_df.iloc[i,-1] = year_df[year_df['ticker'] == ticker]['fiem age']
    elif '2021' in news_df.iloc[i,1]:
        ticker = news_df.iloc[i,4][2:-2]
        news_df.iloc[i,-1] = year_df[year_df['ticker'] == ticker]['fiem age'] - 1
    elif '2020' in news_df.iloc[i,1]:
        ticker = news_df.iloc[i,4][2:-2]
        news_df.iloc[i,-1] = year_df[year_df['ticker'] == ticker]['fiem age'] - 2
    else:
        ticker = news_df.iloc[i,4][2:-2]
        news_df.iloc[i,-1] = year_df[year_df['ticker'] == ticker]['fiem age'] - 3

100%|█████████████████████████████████████| 4561/4561 [00:00<00:00, 4773.58it/s]


In [13]:
news_df['Firm Age'].value_counts()

17      241
18      232
26      206
16      193
25      168
       ... 
64        1
184       1
85        1
129       1
2023      1
Name: Firm Age, Length: 149, dtype: int64

## 4. 트위터 팔로워 추가하기

In [14]:
followers_df = pd.read_excel('/Users/parkjunhyeong/Desktop/박준형/02. 대내 및 대외활동/01. 대내활동/03. Biz&AI 랩/02. 소스/01. 데이터/05. 통제변수 데이터/Twitter Follower Number_ver1.0.xlsx', sheet_name = 'Sheet1', index_col = 0)
followers_df

,ticker,Description,Account,URL,URL.1,Follower
0,CSCO,Cisco Systems Inc,@Cisco,https://twitter.com/Cisco,https://twitter.com/Cisco,743305
1,UAL,United Continental Holdings Inc,@united,https://twitter.com/united,https://twitter.com/united,1174490
2,W,Wayfair Inc Class A,@Wayfair,https://twitter.com/Wayfair,https://twitter.com/Wayfair,77326
3,BA,Boeing,@Boeing,https://twitter.com/Boeing,https://twitter.com/Boeing,668156
4,GILD,Gilead Sciences Inc,@GileadSciences,https://twitter.com/GileadSciences,https://twitter.com/GileadSciences,73710
...,...,...,...,...,...,...
215,DOV,Dover Corp,@DoverCorp,https://twitter.com/DoverCorp,https://twitter.com/DoverCorp,1072
216,M,Macys Inc.,@macysnews,https://twitter.com/macysnews,https://twitter.com/macysnews,1236
217,CRM,Salesforce.Com Inc,@salesforce,https://twitter.com/salesforce,https://twitter.com/salesforce,589923
218,BLK,Blackrock Inc,@BlackRock,https://twitter.com/BlackRock,https://twitter.com/BlackRock,599976


In [15]:
news_df['Twitter Followers'] = 0
for i in tqdm(range(len(news_df))):
    try:
        ticker = news_df.iloc[i,4][2:-2]
        followers = followers_df[followers_df['ticker'] == ticker]['Follower']
        news_df.iloc[i,-1] = followers
    except Exception as e:
        print(i, e)

100%|█████████████████████████████████████| 4561/4561 [00:00<00:00, 6467.50it/s]


In [16]:
news_df

,URL,Publish Date,Title,Main Text,Ticker,Description,News Type,Sentiment,Investor Reaction,GICS Sectors,Firm Age,Twitter Followers
0,https://www.nytimes.com/2019/01/01/business/me...,2019-01-01,Netflix Blocks Show in Saudi Arabia Critical o...,Netflix has blocked an episode of its show “Pa...,['NFLX'],['Netflix Inc'],ESG,Negative,0.064605,Communication Services,23,21667744
1,https://www.nytimes.com/2019/01/02/business/ju...,2019-01-02,Julie Sweet of Accenture Could See Her Future....,Why did you go to law school?\n \n I decided t...,['ACN'],['Accenture Plc Class A'],Non-ESG,Positive,-0.001291,Information Technology,31,551168
2,https://www.nytimes.com/2019/01/02/business/te...,2019-01-02,"Tesla Reports Record Output, but Cuts Prices, ...",Tesla shares ended trading nearly 7 percent lo...,['TSLA'],['Tesla Inc'],Financial Performance,Negative,-0.010247,Consumer Discretionary,17,19694123
3,https://www.nytimes.com/2019/01/03/business/de...,2019-01-03,Bristol-Myers to Acquire Celgene in Deal Worth...,Bristol-Myers Squibb said on Thursday that it ...,['BMY'],['Bristol Myers Squibb'],Financial Performance,Neutral,-0.056534,Health Care,133,157472
4,https://www.nytimes.com/2019/01/03/business/de...,2019-01-03,Was Apple’s Warning a Surprise? Not to Its Sup...,"Apple’s stock fell 10 percent on Thursday, its...",['AAPL'],['Apple Inc'],Financial Performance,Negative,-0.027306,Information Technology,44,9115548
...,...,...,...,...,...,...,...,...,...,...,...,...
4556,https://www.nytimes.com/2022/10/18/technology/...,2022-10-18,Amazon Labor Union Loses Election at Warehouse...,Mr. Smalls said the election “wasn’t free and ...,['AMZN'],['Amazon.Com Inc.'],ESG,Positive,0.010129,Consumer Discretionary,29,5401566
4557,https://www.nytimes.com/2022/10/20/technology/...,2022-10-20,Texas Sues Google for Collecting Biometric Dat...,"José Castañeda, a Google spokesman, said in a ...",['GOOG'],['Alphabet Inc Class C'],ESG,Negative,-0.040082,Communication Services,25,28908576
4558,https://www.nytimes.com/2022/10/25/technology/...,2022-10-25,Microsoft Reports Slowest Revenue Growth in Fi...,"In the most recent quarter, softness in the gl...",['MSFT'],['Microsoft Corp'],Financial Performance,Positive,-0.036752,Information Technology,48,12790258
4559,https://www.nytimes.com/2022/10/27/technology/...,2022-10-27,"Apple’s iPhone Powers Growth, but Signs Point ...",When Apple released the 16th version of its iP...,['AAPL'],['Apple Inc'],Financial Performance,Positive,-0.003742,Information Technology,47,9115548


## 5. Total Asset 추가하기

In [52]:
asset_df = pd.read_excel('/Users/parkjunhyeong/Desktop/박준형/02. 대내 및 대외활동/01. 대내활동/03. Biz&AI 랩/02. 소스/01. 데이터/05. 통제변수 데이터/tickers_vF 복사본.xlsx', sheet_name = 'TA')
asset_df.isna().sum()

Description          0
ticker               0
Total Assets_2017    0
Total Assets_2018    0
Total Assets_2019    0
Total Assets_2020    0
Total Assets_2021    0
Total Assets_2022    0
Measure              0
dtype: int64

In [53]:
news_df['Total Asset'] = 0

for i in tqdm(range(len(news_df))):
    if '2022' in news_df.iloc[i,1]:
        ticker = news_df.iloc[i,4][2:-2]
        news_df.iloc[i,-1] = float(asset_df[asset_df['ticker'] == ticker]['Total Assets_2022'])
    elif '2021' in news_df.iloc[i,1]:
        ticker = news_df.iloc[i,4][2:-2]
        news_df.iloc[i,-1] = float(asset_df[asset_df['ticker'] == ticker]['Total Assets_2021'])
    elif '2020' in news_df.iloc[i,1]:
        ticker = news_df.iloc[i,4][2:-2]
        news_df.iloc[i,-1] = float(asset_df[asset_df['ticker'] == ticker]['Total Assets_2020'])
    else:
        ticker = news_df.iloc[i,4][2:-2]
        news_df.iloc[i,-1] = float(asset_df[asset_df['ticker'] == ticker]['Total Assets_2019'])

100%|█████████████████████████████████████| 4561/4561 [00:00<00:00, 6086.66it/s]


In [55]:
news_df

,URL,Publish Date,Title,Main Text,Ticker,Description,News Type,Sentiment,Investor Reaction,GICS Sectors,Firm Age,Twitter Followers,Total Asset
0,https://www.nytimes.com/2019/01/01/business/me...,2019-01-01,Netflix Blocks Show in Saudi Arabia Critical o...,Netflix has blocked an episode of its show “Pa...,['NFLX'],['Netflix Inc'],ESG,Negative,0.064605,Communication Services,23,21667744,33.976
1,https://www.nytimes.com/2019/01/02/business/ju...,2019-01-02,Julie Sweet of Accenture Could See Her Future....,Why did you go to law school?\n \n I decided t...,['ACN'],['Accenture Plc Class A'],Non-ESG,Positive,-0.001291,Information Technology,31,551168,29.790
2,https://www.nytimes.com/2019/01/02/business/te...,2019-01-02,"Tesla Reports Record Output, but Cuts Prices, ...",Tesla shares ended trading nearly 7 percent lo...,['TSLA'],['Tesla Inc'],Financial Performance,Negative,-0.010247,Consumer Discretionary,17,19694123,34.309
3,https://www.nytimes.com/2019/01/03/business/de...,2019-01-03,Bristol-Myers to Acquire Celgene in Deal Worth...,Bristol-Myers Squibb said on Thursday that it ...,['BMY'],['Bristol Myers Squibb'],Financial Performance,Neutral,-0.056534,Health Care,133,157472,129.940
4,https://www.nytimes.com/2019/01/03/business/de...,2019-01-03,Was Apple’s Warning a Surprise? Not to Its Sup...,"Apple’s stock fell 10 percent on Thursday, its...",['AAPL'],['Apple Inc'],Financial Performance,Negative,-0.027306,Information Technology,44,9115548,338.516
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4556,https://www.nytimes.com/2022/10/18/technology/...,2022-10-18,Amazon Labor Union Loses Election at Warehouse...,Mr. Smalls said the election “wasn’t free and ...,['AMZN'],['Amazon.Com Inc.'],ESG,Positive,0.010129,Consumer Discretionary,29,5401566,462.675
4557,https://www.nytimes.com/2022/10/20/technology/...,2022-10-20,Texas Sues Google for Collecting Biometric Dat...,"José Castañeda, a Google spokesman, said in a ...",['GOOG'],['Alphabet Inc Class C'],ESG,Negative,-0.040082,Communication Services,25,28908576,365.264
4558,https://www.nytimes.com/2022/10/25/technology/...,2022-10-25,Microsoft Reports Slowest Revenue Growth in Fi...,"In the most recent quarter, softness in the gl...",['MSFT'],['Microsoft Corp'],Financial Performance,Positive,-0.036752,Information Technology,48,12790258,364.840
4559,https://www.nytimes.com/2022/10/27/technology/...,2022-10-27,"Apple’s iPhone Powers Growth, but Signs Point ...",When Apple released the 16th version of its iP...,['AAPL'],['Apple Inc'],Financial Performance,Positive,-0.003742,Information Technology,47,9115548,352.755


## 6. ROA 추가하기

In [56]:
roa_df = pd.read_excel('/Users/parkjunhyeong/Desktop/박준형/02. 대내 및 대외활동/01. 대내활동/03. Biz&AI 랩/02. 소스/01. 데이터/05. 통제변수 데이터/Osiris_Export_3 복사본.xlsx', sheet_name = 'Sheet1', index_col = 0)
roa_df.isna().sum()


ticker         0
Description    0
2022 ROA       1
2021 ROA       1
2020 ROA       1
2019 ROA       1
2018 ROA       1
2017 ROA       1
dtype: int64

In [57]:
roa_df = roa_df.fillna(roa_df.mean())

/var/folders/z6/nn0fl27x39547ybzc5y11s8m0000gn/T/ipykernel_5183/3022352208.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  roa_df = roa_df.fillna(roa_df.mean())


In [58]:
roa_df

,ticker,Description,2022 ROA,2021 ROA,2020 ROA,2019 ROA,2018 ROA,2017 ROA
0,CSCO,Cisco Systems Inc,15.40,13.60,14.73,14.90,11.99,9.46
1,UAL,United Continental Holdings Inc,1.47,-3.75,-14.81,7.44,5.40,7.14
2,W,Wayfair Inc Class A,-36.84,-2.84,4.49,-33.24,-26.55,-20.12
3,BA,Boeing,-3.66,-3.63,-9.52,-1.69,9.89,9.00
4,GILD,Gilead Sciences Inc,9.20,12.18,2.44,8.37,12.25,19.25
...,...,...,...,...,...,...,...,...
215,DOV,Dover Corp,11.82,13.46,9.20,9.72,8.67,8.22
216,M,Macys Inc.,9.00,10.61,-27.05,3.44,7.40,7.74
217,CRM,Salesforce.Com Inc,0.67,1.61,3.86,1.28,3.20,1.91
218,BLK,Blackrock Inc,5.35,5.35,3.69,3.43,3.37,2.40


In [59]:
news_df['ROA'] = 0

for i in tqdm(range(len(news_df))):
    if '2022' in news_df.iloc[i,1]:
        ticker = news_df.iloc[i,4][2:-2]
        news_df.iloc[i,-1] = roa_df[roa_df['ticker'] == ticker]['2022 ROA']
    elif '2021' in news_df.iloc[i,1]:
        ticker = news_df.iloc[i,4][2:-2]
        news_df.iloc[i,-1] = roa_df[roa_df['ticker'] == ticker]['2021 ROA']
    elif '2020' in news_df.iloc[i,1]:
        ticker = news_df.iloc[i,4][2:-2]
        news_df.iloc[i,-1] = roa_df[roa_df['ticker'] == ticker]['2020 ROA']
    else:
        ticker = news_df.iloc[i,4][2:-2]
        news_df.iloc[i,-1] = roa_df[roa_df['ticker'] == ticker]['2019 ROA']

100%|█████████████████████████████████████| 4561/4561 [00:00<00:00, 5617.01it/s]


In [61]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4561 entries, 0 to 4560
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   URL                4561 non-null   object 
 1   Publish Date       4561 non-null   object 
 2   Title              4561 non-null   object 
 3   Main Text          4559 non-null   object 
 4   Ticker             4561 non-null   object 
 5   Description        4561 non-null   object 
 6   News Type          4561 non-null   object 
 7   Sentiment          4561 non-null   object 
 8   Investor Reaction  4561 non-null   float64
 9   GICS Sectors       4561 non-null   object 
 10  Firm Age           4561 non-null   int64  
 11  Twitter Followers  4561 non-null   int64  
 12  Total Asset        4561 non-null   float64
 13  ROA                4561 non-null   float64
dtypes: float64(3), int64(2), object(9)
memory usage: 499.0+ KB


In [83]:
news_df.drop('esg_score', axis = 1, inplace = True)

In [84]:
news_df.to_csv('/Users/parkjunhyeong/Desktop/박준형/02. 대내 및 대외활동/01. 대내활동/03. Biz&AI 랩/02. 소스/01. 데이터/06. 회귀분석 데이터/Composed_Data_ver1.2.csv')
